In [2]:
from flask import Flask, render_template, Response
import cv2
import face_recognition
import numpy as np

In [3]:
app = Flask(__name__)
camera = cv2.VideoCapture(0)


Rohit_image = face_recognition.load_image_file("Rohit/Rohit.jpg")
Rohit_face_encoding = face_recognition.face_encodings(Rohit_image)[0]

Kohli_image = face_recognition.load_image_file("Kohli/Kohli.jpeg")
Kohli_face_encoding = face_recognition.face_encodings(Kohli_image)[0]

known_face_encodings = [
    Rohit_face_encoding,
    Kohli_face_encoding
]

known_face_names = [
    "Rohit",
    "Kohli"
]
face_locations = []
face_encodings = []
face_names = []
def generate_frames():  
    while True:
        success, frame = camera.read() 
        if not success:
            break
        else:
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            rgb_small_frame = small_frame[:, :, ::-1]           
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
            face_names = []
            for face_encoding in face_encodings:
                # Compare a list of face encodings against a candidate encoding to see if they match.
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                 # Given a list of face encodings, compare them to a known face encoding and get a euclidean distance for each comparison face. 
                # The distance tells you how similar the faces are.
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                # Returns the indices of the minimum values along an axis.
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
                face_names.append(name)
            

            for (top, right, bottom, left), name in zip(face_locations, face_names):
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


@app.route('/')
def Display():
    return render_template('Display.html')
@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')
app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Sep/2021 15:27:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2021 15:27:38] "GET /video_feed HTTP/1.1" 200 -
